In [7]:
import os
import langchain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, ConversationChain
from langchain.callbacks import get_openai_callback
from langchain.memory import ConversationBufferWindowMemory, CombinedMemory, ConversationSummaryMemory, ChatMessageHistory
from langchain import hub
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.ai import AIMessage
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
from langchain_community.utilities.openweathermap import OpenWeatherMapAPIWrapper
from langchain.agents import AgentExecutor, create_react_agent, Tool
import langdetect
from os import environ
from dotenv import load_dotenv

load_dotenv(dotenv_path='/home/knguyen02311/Datadisk_link/Programming/Python/AIBots/backend/.env.dev', verbose=True)

True

### Requirements:
- Introduce local tourist attractions
- Set up travel plans from user requirements (duration, location, number of visitors, ...)
- Recommend places to visit, accommodations, food & beverage locations in surrounding areas

In [3]:
from googleapiclient.discovery import build

my_api_key = os.environ["GOOGLE_API_KEY"]  #The API_KEY you acquired
my_cse_id = os.environ["GOOGLE_CSE_ID"]  #The search-engine-ID you created


def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res['items']


results = google_search('tourist attractions in Hanoi', my_api_key, my_cse_id, num=3)
for result in results:
    print(result)

{'kind': 'customsearch#result', 'title': 'THE 15 BEST Things to Do in Hanoi - 2024 (with Photos) - Tripadvisor', 'htmlTitle': 'THE 15 BEST Things to Do in <b>Hanoi</b> - 2024 (with Photos) - Tripadvisor', 'link': 'https://www.tripadvisor.com/Attractions-g293924-Activities-Hanoi.html', 'displayLink': 'www.tripadvisor.com', 'snippet': 'Top Attractions in Hanoi · 1. Old Quarter · 28,578. Neighborhoods · 2. Lake of the Restored Sword (Hoan Kiem Lake) · 13,664. Bodies of Water · 3. Temple of\xa0...', 'htmlSnippet': 'Top <b>Attractions in Hanoi</b> &middot; 1. Old Quarter &middot; 28,578. Neighborhoods &middot; 2. Lake of the Restored Sword (Hoan Kiem Lake) &middot; 13,664. Bodies of Water &middot; 3. Temple of&nbsp;...', 'formattedUrl': 'https://www.tripadvisor.com/Attractions-g293924-Activities-Hanoi.html', 'htmlFormattedUrl': 'https://www.tripadvisor.com/<b>Attractions</b>-g293924-Activities-<b>Hanoi</b>.html', 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn3.gstatic.com/imag

In [8]:
search = GoogleSearchAPIWrapper(k=5)
weather = OpenWeatherMapAPIWrapper(openweathermap_api_key=os.environ["OPENWEATHERMAP_API_KEY"])

tools = [
    Tool(
        name="google_search",
        description="Search Google for recent results.",
        func=search.run),
    Tool(
        name="weather",
        description="Get weather information.",
        func=weather.run)]

In [5]:
search.results("tourist locations in Hanoi", 3)

[{'title': 'THE 15 BEST Things to Do in Hanoi - 2024 (with Photos) - Tripadvisor',
  'link': 'https://www.tripadvisor.com/Attractions-g293924-Activities-Hanoi.html',
  'snippet': 'Top Attractions in Hanoi · 1. Old Quarter · 28,578. Neighborhoods · 2. Lake of the Restored Sword (Hoan Kiem Lake) · 13,664. Bodies of Water · 3. Temple of\xa0...'},
 {'title': '16 Top-Rated Tourist Attractions in Hanoi | PlanetWare',
  'link': 'https://www.planetware.com/vietnam/hanoi-vie-1-9.htm',
  'snippet': 'Dec 27, 2023 ... 16 Top-Rated Tourist Attractions in Hanoi · 1. Explore Hanoi Old Town Quarter · 2. Stroll around Hoan Kiem Lake · 3. Admire the Temple of\xa0...'},
 {'title': '11 must-see attractions in Ha Noi | Vietnam Tourism',
  'link': 'https://vietnam.travel/things-to-do/11-must-see-attractions-ha-noi',
  'snippet': "Saint Joseph's Cathedral ... The cafes by the Cathedral are a perfect spot to watch the bustle of Hanoi. Saint Joseph's Cathedral (also known as Hanoi Cathedral),\xa0..."}]

In [17]:
weather.run("Huế, VN")

'In Huế, VN, the current weather is as follows:\nDetailed status: light intensity drizzle\nWind speed: 4.12 m/s, direction: 270°\nHumidity: 94%\nTemperature: \n  - Current: 21.06°C\n  - High: 21.06°C\n  - Low: 21.06°C\n  - Feels like: 21.68°C\nRain: {}\nHeat index: None\nCloud cover: 75%'

In [13]:
prompt = hub.pull("hwchase17/react")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [18]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=1,
    openai_api_key=os.environ["OPENAI_API_KEY"],
    max_tokens=2048
)
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

[]

In [6]:
agent_executor.invoke({"input": "Gợi ý cho tôi một vài món ăn, cũng như quán ăn ngon giá rẻ ở Huế"})



> Entering new AgentExecutor chain...
I should search for popular local dishes and affordable restaurants in Hue.
Action: google_search
Action Input: popular local dishes in HueFamous Food Locals Love to Eat in Hue · Bun bo hue · Nem lui · Com hen · Banh khoai · Banh beo · Banh loc goi · Bun thit nuong · Va tron salad; Che ... 3.2. Best food in Hue? Com Hen (Clam Rice) ... Calm Rice is a combination of rice, clams and local spices. This simple dish is one of the best Hue foods that can ... More videos on YouTube · 1.The number one Hue specialty dish is Hue beef noodle soup which called BÚN BÒ HUẾ in Vietnamese · 2.The second on top of Hue ... Dec 9, 2013 ... Favorite eats in Hue · Banh Nam · Banh Beo · Bun Bo Hue · Banh Khoai (Hue pancakes) · Banh Ram It! · Have you tried Hue food? What's your favorite ... Oct 6, 2020 ... Clam rice or clam noodles is an iconic Hue food. A simple dish with humble origins, it started as a way to use up leftover rice. It generally ...I should now search

In [16]:
agent_executor.invoke({"input": "Ngày mai liệu có phù hợp để đi chơi ở Huế không ?"})



> Entering new AgentExecutor chain...
I should check the weather in Huế for tomorrow to see if it's suitable for outdoor activities.
Action: weather
Action Input: Huế tomorrow

NotFoundError: Unable to find the resource

In [ ]:
# import requests
# 
# 
# def weather_search(location: str, search_type: str, date: str | None = None):
#     response = requests.get(f"https://api.openweathermap.org/data/3.0/onecall/day_summary?lat={lat}&lon={lon}&date={date}&appid={API key}")
#     print(response.json())